# Aula: O "Jeito Profissional" de Usar Deep Learning (Transfer Learning)

**A Ponte:** Treinar uma CNN do zero é demorado e exige muitos dados. Na prática, usamos modelos que já foram treinados por gigantes (como Google) em datasets massivos (como o ImageNet, com 14 milhões de imagens) e os adaptamos para o nosso problema.

In [1]:
import sys

# Este comando imprime o caminho exato para o executável Python
# que está rodando este notebook (o "kernel").
print(sys.executable)

/Users/leonardoraiz/Documents/Leonardo/FATEC/Aulas/IA/Outros/Aula 04 - Sistemas Especialistas /.venv/bin/python


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds  # Biblioteca do TF para carregar datasets
import matplotlib.pyplot as plt

/Users/leonardoraiz/Documents/Leonardo/FATEC/Aulas/IA/Outros/Aula 04 - Sistemas Especialistas /.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/leonardoraiz/Documents/Leonardo/FATEC/Aulas/IA/Outros/Aula 04 - Sistemas Especialistas /.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Passo 1: Carregar os Dados e o Modelo Pré-Treinado

Vamos usar o dataset "cats_vs_dogs" e o modelo MobileNetV2, pré-treinado no ImageNet.

In [3]:
# Carregar o dataset 'cats_vs_dogs' do TensorFlow Datasets
(train_ds, validation_ds, test_ds), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

IMG_SIZE = (160, 160) # MobileNetV2 gosta desse tamanho

# Função para redimensionar e normalizar as imagens
def formatar_imagem(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32)
    image = (image / 127.5) - 1  # MobileNet espera pixels de -1 a 1
    return image, label

train_batches = train_ds.map(formatar_imagem).batch(32).prefetch(1)
validation_batches = validation_ds.map(formatar_imagem).batch(32).prefetch(1)

# Carregar o "Cérebro" Pré-Treinado (MobileNetV2)
# include_top=False significa: "Não traga a camada final de classificação do ImageNet"
base_model = keras.applications.MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    weights='imagenet'
)

# Congelar o cérebro!
base_model.trainable = False

print("Modelo base (MobileNetV2) carregado e congelado.")
base_model.summary()

2025-11-03 14:19:07.064850: W external/local_tsl/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]
Generating train examples...:  67%|██████▋   | 15525/23262 [00:02<00:00, 7857.42 examples/s]WARNING:absl:1738 images were corrupted and were skipped

                                                                                            
Shuffling /Users/leonardoraiz/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteX7S0WB/cats_vs_dogs-train.tfrecord*...:   0%|          | 0/23262 [00:00<?, ? examples/s]
Shuffling /Users/leonardoraiz/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteX7S0WB/cats_vs_dogs-train.tfrecord*...:  13%|█▎        | 2941/23262 [00:00<00:00, 29405.96 examples/s]
Shuffling /Users/leonardoraiz/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteX7S0WB/cats_vs_dogs-train.tfrecord*...:  25%|██▌       | 5882/23262 [00:00<00:00, 28475.39 examples/s]
Shuffling /Users/leonardoraiz/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteX7S0WB/cats_vs_dogs-train.tfrecord*...:  38%|███▊      | 8732/23262 [00:00<00:00, 23012.50 examples/s]
Shu

Dataset cats_vs_dogs downloaded and prepared to /Users/leonardoraiz/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Modelo base (MobileNetV2) carregado e congelado.


Model: "mobilenetv2_1.00_160"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 80, 80,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 80, 80,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 80, 80,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 80, 80,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 80, 80,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 80, 80,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 80, 80,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 80, 80,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 81, 81,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 40, 40,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 40, 40,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 40, 40,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

### Passo 2: Construir nosso Modelo "Topo" (Fine-Tuning)

Vamos empilhar nosso classificador em cima da MobileNetV2.

In [6]:
model = keras.Sequential([
    # 1. O "cérebro" congelado como a base
    base_model,
    
    # 2. "Resumir" as features de alto nível que o cérebro nos deu
    layers.GlobalAveragePooling2D(),
    
    # 3. Adicionar uma camada de "Dropout" para evitar overfitting
    layers.Dropout(0.2),
    
    # 4. Nossa camada de decisão final (1 neurônio - Gato ou Cachorro)
    layers.Dense(1, activation='sigmoid') # Sigmoid para classificação binária
])

model.summary()

# Compilar o modelo
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_160            │ (None, 5, 5, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

### Passo 3: Treinar (Muito Rápido!) e Avaliar

Note como vamos treinar por poucas épocas e alcançar uma acurácia altíssima.

In [5]:
print("Iniciando o Transfer Learning...")

# Note que estamos treinando apenas os "parâmetros treináveis" 
# que são as poucas camadas que adicionamos, e não a MobileNet inteira.

history = model.fit(train_batches, 
                    epochs=5, 
                    validation_data=validation_batches)

print("\nTreinamento Concluído!")

# Avaliar no dataset de teste
test_loss, test_accuracy = model.evaluate(test_ds.map(formatar_imagem).batch(32))
print(f"\nAcurácia no teste (dados inéditos): {test_accuracy * 100:.2f}%")

# (Os alunos ficarão impressionados com uma acurácia de 98%+ em 5 épocas,
# algo impossível para o modelo do CIFAR-10)

Iniciando o Transfer Learning...
Epoch 1/5


2025-11-03 14:19:32.356565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


196/582 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.5852 - loss: 0.7089

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9


229/582 ━━━━━━━━━━━━━━━━━━━━ 15s 43ms/step - accuracy: 0.6071 - loss: 0.6828

...

582/582 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9798 - loss: 0.0633

Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9


582/582 ━━━━━━━━━━━━━━━━━━━━ 29s 50ms/step - accuracy: 0.9798 - loss: 0.0633 - val_accuracy: 0.9798 - val_loss: 0.0576

Treinamento Concluído!
27/73 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.9912 - loss: 0.0527

Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9


73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 85ms/step - accuracy: 0.9865 - loss: 0.0569

Acurácia no teste (dados inéditos): 98.37%
